In your report, evaluate all three models and decide on your best. Be clear about the decisions you made that led to these models (feature selection, regularization parameter selection, model evaluation criteria) and why you think that particular model is the best of the three. Also reflect on the strengths and limitations of regression as a modeling approach. Were there things you couldn't do but you wish you could have done?

In [213]:
import math
import warnings

from IPython.display import display
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import scipy
import sklearn
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
import statsmodels.formula.api as smf

# Display preferences.
%matplotlib inline
pd.options.display.float_format = '{:.3f}'.format
pd.options.display.max_colwidth = 100
pd.options.display.max_rows = 100



# Suppress annoying harmless error.
warnings.filterwarnings(
    action="ignore",
    module="scipy",
    message="^internal gelsd"
)

# Non-DataScience/ML packages
import re
import string

In [214]:
# Test your model with different holdout groups.
def run_with_the_holdouts(fit_type, data, target, holdout_percent):
    from sklearn.model_selection import train_test_split
    # Use train_test_split to create the necessary training and test groups
    X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=holdout_percent, random_state=20)
    print('With {0:.0%} Holdout: '.format(holdout_percent*1) + str(fit_type.fit(X_train, y_train).score(X_test, y_test)))
    print('Testing on Sample: ' + str(fit_type.fit(data, target).score(data, target)))

In [215]:
def run_the_cross_validations(fit_type, data, target):
    from sklearn.model_selection import cross_val_score
    return cross_val_score(bnb, data, target, cv=10)

In [216]:
path="../../../../Datafiles/"
file = 'table_8_offenses_known_to_law_enforcement_new_york_by_city_2013.xls'
df = pd.read_excel(path + file, header=4)
print("there are {} entries in the data frame".format(len(df)))

there are 351 entries in the data frame


In [217]:
# Let's clean up the column names
colnames = df.columns
newcolnameslist = []

for colname in colnames:
    newcolname=colname.replace('\n','').replace(' ','').capitalize() # strip out newlines, spaces, and captitalize
    newcolname=re.sub(r'\([^)]*\)', '', newcolname)                  # remove parenthesized stuff
    newcolnameslist.append(newcolname)                               # put all the column names into a list 

df.columns = newcolnameslist
print(newcolnameslist)

['City', 'Population', 'Violentcrime', 'Murderandnonnegligentmanslaughter', 'Rape1', 'Rape2', 'Robbery', 'Aggravatedassault', 'Propertycrime', 'Burglary', 'Larceny-theft', 'Motorvehicletheft', 'Arson3']


In [218]:
def return_diff_min_max_cv_score(cv_in):
    print("min cv_score={0:.4%}, max={1:.4%}, mean={3:.4%}, and delta is {2:.4%}".format(cv_in.min(),
                                                                      cv_in.max(), 
                                                                      cv_in.max()-cv_in.min(),
                                                                      cv_in.mean()))

In [219]:
df[df['City'].str.contains('figures shown')]
df[df['City'].str.contains('publish')]

df = df[(df['City'].str.contains('figures shown') == False)] # We don't want any documentation rows
df = df[(df['City'].str.contains('publish') == False)]       # We don't want any other documentation rows

In [220]:
df['PopulationSquared'] = df.Population.pow(2)
df.drop('Rape1', 1, inplace=True)
df['Robbery'] = np.where(df['Robbery']>=1, 1, 0.0)
df = df.rename(columns={'Arson3': 'Arson', 'Rape2': 'Rape', 'Aggravatedassault': 'AggravatedAssault',
                        'Violentcrime':'ViolentCrime','Propertycrime': 'PropertyCrime', 
                        'Larceny-theft':'LarcenyTheft','Motorvehicletheft':'MotorVehicleTheft'})
# df.Arson = df.Arson.astype(int) # Let's make it an int
df['Murder'] = np.where(df['Murderandnonnegligentmanslaughter']>=1, 1, 0.0)
df.Arson.fillna(0, inplace=True) # Let's clean up Arson3

In [221]:
df.head(5)
# df.describe()

,City,Population,ViolentCrime,Murderandnonnegligentmanslaughter,Rape,Robbery,AggravatedAssault,PropertyCrime,Burglary,LarcenyTheft,MotorVehicleTheft,Arson,PopulationSquared,Murder
0,Adams Village,1861.000,0.000,0.000,0.000,0.000,0.000,12.000,2.000,10.000,0.000,0.000,3463321.000,0.000
1,Addison Town and Village,2577.000,3.000,0.000,0.000,0.000,3.000,24.000,3.000,20.000,1.000,0.000,6640929.000,0.000
2,Akron Village,2846.000,3.000,0.000,0.000,0.000,3.000,16.000,1.000,15.000,0.000,0.000,8099716.000,0.000
3,Albany,97956.000,791.000,8.000,30.000,1.000,526.000,4090.000,705.000,3243.000,142.000,0.000,9595377936.000,1.000
4,Albion Village,6388.000,23.000,0.000,3.000,1.000,16.000,223.000,53.000,165.000,5.000,0.000,40806544.000,0.000


In [222]:
# Let's normalize/preprocess the data frame to get them all on the same scale
df_city = df['City'] # we need to drop all string columns to be able to do Pandas preprocessing
df.drop('City', 1, inplace=True)
names = df.columns
# print("names[1:]={}".format(names[1:]))
# names = ['Population', 'ViolentCrime', 'Rape', 'AggravatedAssault', 'PropertyCrime', 'Burglary','LarcenyTheft',
#                                                'MotorVehicleTheft', 'Arson']
# print('testing',df.loc[:,'Population':])
# print(names[2:])
df =  pd.DataFrame(preprocessing.scale(df), columns=names)
# df =  pd.merge(df_city, df, left_index=True, right_index=True) # Let's add the cit back into df

/Users/lou/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:180: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


In [223]:
# print(df.head(20))

In [224]:
# Warning above -> may be some NaN's or infinities here... watch the range of values.  Look for anomalies.

In [225]:
num_columns = ['Population', 'ViolentCrime', 'Rape', 'AggravatedAssault', 'PropertyCrime', 'Burglary','LarcenyTheft',
                                               'MotorVehicleTheft', 'Arson']

In [226]:
# print("Rape mean = {}".format(df['Rape'].mean))

In [227]:
# Let's create some new features
df['Arson_Burglary'] = df['Arson'] * df['Burglary']
df['ViolentCrime_MotorVehicleTheft'] = df['ViolentCrime'] * df['MotorVehicleTheft']
df['LarcenyTheft_Arson'] = np.log2(df['LarcenyTheft'] * df['Arson'])
df['Arson_Pop'] = df['Arson'] * df['Population']
df['MotorVehicleTheft_Arson'] = df['MotorVehicleTheft'] / df['Arson']
df['PropertyCrime_VehicleTheft'] = df['PropertyCrime'] * df['MotorVehicleTheft'] * 100
df.fillna(df.mean()['ViolentCrime_MotorVehicleTheft'], inplace=True) # fill in missing values
df['PropertyCrime_binary'] = np.where(df['PropertyCrime'] >.50, 1,0)
df['MotorVehicleTheft_binary'] = np.where(df['MotorVehicleTheft'] >.50, 1,0)
df['Rape_binary'] = np.where(df['Rape'].abs() > df['Rape'].abs().mean(),1,0)
df.drop('Rape', inplace=True, axis=1)
# df.drop('City', inplace=True, axis=1)

/Users/lou/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in log2
  after removing the cwd from sys.path.


In [228]:
# check the same thing as problem #1 above...

In [229]:
# df.isnull().sum()
# df[['Rape','Rape_binary']].sample(100)
# df.describe()


In [230]:
# Define the training and test sizes.
trainsize = int(df.shape[0] / 2)
df_test = df.iloc[trainsize:, :]
df_train = df.iloc[:trainsize, :]

In [231]:
# df_train['Rape_binary'].head(300)
# df_train.head(50)

### Vanilla Logistic Regression

In [232]:
# Train the model first
# Set up the regression model to predict defaults using all other
# variables as features.
lr = LogisticRegression(C=1e20, solver='lbfgs')

# Y = df_train['PropertyCrime'].values.reshape(-1,1)
# X = df_train[['LarcenyTheft', 'MotorVehicleTheft','Rape','AggravatedAssault']]
df_train.shape
label_column = ['Rape_binary']
# Y_train = df_train[label_column].values.reshape(-1, 1)
y_train = df_train[label_column].values.reshape(-1, 1).ravel()
X_train = df_train.loc[:, ~(df_train.columns).isin(label_column)]

y_test = df_test[label_column].values.reshape(-1, 1).ravel() # ravel() was added
X_test = df_test.loc[:, ~(df_test.columns).isin(label_column)]

# X_train = df_train[['LarcenyTheft']]
lr.fit(X_train, y_train)
print('\nR-squared simple model training set yields:')
print(lr.score(X_train, y_train))
print("here comes the test set")
print(lr.score(X_test, y_test))


R-squared simple model training set yields:
1.0
here comes the test set
0.9540229885057471


In [233]:
# error above - something is changing soon.
# issue #2 above, columns are expecting a list.  Change 

In [234]:
X_train.columns

Index(['Population', 'ViolentCrime', 'Murderandnonnegligentmanslaughter',
       'Robbery', 'AggravatedAssault', 'PropertyCrime', 'Burglary',
       'LarcenyTheft', 'MotorVehicleTheft', 'Arson', 'PopulationSquared',
       'Murder', 'Arson_Burglary', 'ViolentCrime_MotorVehicleTheft',
       'LarcenyTheft_Arson', 'Arson_Pop', 'MotorVehicleTheft_Arson',
       'PropertyCrime_VehicleTheft', 'PropertyCrime_binary',
       'MotorVehicleTheft_binary'],
      dtype='object')

In [235]:
# This is Vanilla Logistic Regression
# Declare a logistic regression classifier.
# Parameter regularization coefficient C described above.

# Fit the model using the training set.
fit = lr.fit(X_train, y_train)

# Display.
print('Coefficients')
print(fit.coef_)
print(fit.intercept_)
pred_y_sklearn = lr.predict(X_test)

print('\n Accuracy by admission status')
print(pd.crosstab(pred_y_sklearn, y_test))

print('\n Percentage accuracy')
print(lr.score(X_test, y_test))

test
Coefficients
[[-4.91346200e+02  2.20944037e+02 -2.16354506e+02 -1.53308025e+02
   3.05223298e+02 -2.00560200e+02  5.25393399e+02 -2.97520775e+02
  -3.32441624e+02 -9.00331800e+02  1.33055708e+01  2.10498009e+01
  -4.39555813e+01 -7.37137271e+00  3.15452049e+02  9.06395126e+01
   3.28563679e+01  7.35921283e+01  4.13081295e-01  4.13081295e-01]]
[-387.55144442]

 Accuracy by admission status
col_0    0  1
row_0        
0      163  5
1        3  3

 Percentage accuracy
0.9540229885057471


### Ridge Logistic Regression

In [242]:
# Ridge Logistic Regression
# Fitting a ridge regression model. Alpha is the regularization
# parameter (usually called lambda). As alpha gets larger, parameter
# shrinkage grows more pronounced. Note that by convention, the
# intercept is not regularized. Since we standardized the data
# earlier, the intercept should be equal to zero and can be dropped.

ridgeregr = linear_model.Ridge(alpha=0.0010, fit_intercept=False) 
ridgeregr.fit(X_train, Y_train)
print(ridgeregr.score(X_train, Y_train))
origparams = ridgeregr.coef_[0]
print(origparams)

ridgeregrBig = linear_model.Ridge(alpha=0.010, fit_intercept=False)
ridgeregrBig.fit(X_train, Y_train)
print(ridgeregrBig.score(X_train, Y_train))

0.7218428702257963
-0.5274197596583162
0.6936458768375031


### Lasso Regression

In [244]:
# Lasso Regression
# Small number of parameters.
lass = linear_model.Lasso(alpha=0)
lassfit = lass.fit(X_train, Y_train)
print('R² for the model with few features:')
print(lass.score(X_train, Y_train))


R² for the model with few features:
0.7478712753900834


/Users/lou/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  after removing the cwd from sys.path.
/Users/lou/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:478: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  positive)
/Users/lou/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


### Writeup  
In your report, evaluate all three models and decide on your best. Be clear about the decisions you made that led to these models (feature selection, regularization parameter selection, model evaluation criteria) and why you think that particular model is the best of the three. Also reflect on the strengths and limitations of regression as a modeling approach. Were there things you couldn't do but you wish you could have done?

I chooose the vanilla logistic regression, because it provided the highest R² value.